# Register Agent to Agentspace

@wangdave

### Prerequisites

- You have an Agentspace application deployed
- You have Agent Engines deployed
- Create a .env file (see below)
- You create OAuth 2.0 client with proper scopes and redirect URL:
https://vertexaisearch.cloud.google.com/oauth-redirect

Example below is based on [Google's OAuth 2.0 client](https://support.google.com/googleapi/answer/6158849):
  - From the GCP console, search for "OAuth consent screen"
  - Create an OAuth 2.0 client for Web application
  - Add the redirect URL specified above
  - Click **Create**
  - Download the `client_secret.json` and upload it here

For other 3rd party OAuth clients, follow their documentation to obtain authorization information.


### 0. Create .env if you don't have one
Fill in the following values in the `.env` file:

In [ ]:
from dotenv import load_dotenv
import IPython
import google_auth_oauthlib.flow
import json
import os
import vertexai


In [ ]:
%%writefile .env


# Your Google Cloud project ID.
PROJECT_ID=""

# Your Google Cloud project number.
PROJECT_NUMBER=""

# The location of your AgentSpace engine (e.g., "global", "us-central1").
LOCATION="global"

# The GCS bucket to use for storing logs and other data.
GCS_BUCKET="gs://"

# The full resource name of your AgentSpace engine.
AS_APP="enterprise-search-"

# Auth ID for your AgentSpace engine.
AUTH_ID="my-oauth"

# The full resource name of your Reasoning Engine.
REASONING_ENGINE = ""

# The display name for your agent in AgentSpace.
DISPLAY_NAME="A2A Agent Demo"

# A brief description (subtitle) of your agent.
DESCRIPTION="Multi-agent using A2A, ADK, Agent Engine and remote MCP server"

# The tool description that defines the agent's capabilities and behavior.
TOOL_DESCRIPTION="***REMEMBER ALWAYS USE THIS TOOL TO ANSWER EVERY QUESTION***. You're an export of weather and cocktail, answer questions regarding weather and cocktail information. You can answer questions like: 1) What is the weather in SF, CA today? 2) List a random cocktail. 3) What is the ID of margarita? 3) What're the ingredients for a Mojito? """

# The unique identifier for your agent.
AGENT_ID="a2a-agent-engine-host"

### Install required libraries

In [ ]:
!pip3 install -qq -U google-adk "google-cloud-aiplatform[agent_engines]" google-cloud-discoveryengine google-auth-oauthlib

### Restart Kernel

In [ ]:

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

## 1. Load Environment Variables and Initialize Vertex AI


In [ ]:
  # take environment variables from .env.
load_dotenv()
PROJECT_ID = os.getenv("PROJECT_ID")
LOCATION = os.getenv("LOCATION", "us-central1")
GCS_BUCKET = os.getenv("GCS_BUCKET")
PROJECT_NUMBER = os.getenv("PROJECT_NUMBER")
REASONING_ENGINE = os.getenv("REASONING_ENGINE")
AS_APP = os.getenv("AS_APP")
AUTH_ID = os.getenv("AUTH_ID")

os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "1"
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
# os.environ["GOOGLE_CLOUD_LOCATION"] = LOCATION

In [ ]:
vertexai.init(
    project=PROJECT_ID,
    location=LOCATION,
    staging_bucket=GCS_BUCKET,
)

In [ ]:
# your agent engine resource name
REASONING_ENGINE

## 2. Generate OAuth 2.0 Authorization URL

In [ ]:
# Required, call the from_client_secrets_file method to retrieve the client ID from a
# client_secret.json file. The client ID (from that file) and access scopes are required. (You can
# also use the from_client_config method, which passes the client configuration as it originally
# appeared in a client secrets file but doesn't access the file itself.)
flow = google_auth_oauthlib.flow.Flow.from_client_secrets_file('client_secret.json',
    scopes=['https://www.googleapis.com/auth/drive.metadata.readonly',
            'https://www.googleapis.com/auth/calendar.readonly',
            'https://www.googleapis.com/auth/bigquery'])

# Required, indicate where the API server will redirect the user after the user completes
# the authorization flow. The redirect URI is required. The value must exactly
# match one of the authorized redirect URIs for the OAuth 2.0 client, which you
# configured in the API Console. If this value doesn't match an authorized URI,
# you will get a 'redirect_uri_mismatch' error.
flow.redirect_uri = 'https://vertexaisearch.cloud.google.com/oauth-redirect'

# Generate URL for request to Google's OAuth 2.0 server.
# Use kwargs to set optional request parameters.
authorization_url, state = flow.authorization_url(
    # Recommended, enable offline access so that you can refresh an access token without
    # re-prompting the user for permission. Recommended for web server apps.
    access_type='offline',
    # Optional, enable incremental authorization. Recommended as a best practice.
    include_granted_scopes='true',
    # Optional, if your application knows which user is trying to authenticate, it can use this
    # parameter to provide a hint to the Google Authentication Server.
    login_hint='hint@example.com',
    # Optional, set prompt to 'consent' will prompt the user for consent
    prompt='consent')
print('OAUTH_AUTH_URI:')
print(authorization_url)

In [ ]:
authorization_url

In [ ]:
os.environ["OAUTH_AUTH_URI"] = authorization_url


## 3. Get OAuth 2.0 Client ID and Secret
- Get `OAUTH_CLIENT_ID` and `OAUTH_CLIENT_SECRET` from the OAuth client setup in the client_secret.json file
- Ensure that `OAUTH_TOKEN_URI` is what your service requires. Example used here is for Google APIs.

In [ ]:

def read_json_file(filepath):
    """
    Reads a JSON file from the specified filepath and returns its content.

    Args:
        filepath (str): The path to the JSON file.

    Returns:
        dict or list: The parsed JSON data, or None if an error occurs.
    """
    try:
        # Open the file in read mode ('r')
        with open(filepath, 'r') as file:
            # Load the JSON content from the file
            data = json.load(file)
            return data
    except FileNotFoundError:
        print(f"Error: The file at '{filepath}' was not found.")
        return None
    except json.JSONDecodeError:
        print(f"Error: The file at '{filepath}' is not a valid JSON file.")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

Note: make sure your `client_secret.json` file is in the same directory as this notebook, and has the following schema, at least has client_id and client_secret:
```json
{
  "web": {
    "client_id": "String",
    "project_id": "String",
    "auth_uri": "String (URL)",
    "token_uri": "String (URL)",
    "auth_provider_x509_cert_url": "String (URL)",
    "client_secret": "String",
    "redirect_uris": "Array<String (URL)>"
  }
}
``` 

If you don't have a valid `client_secret.json` file, you can manually input the client ID and client secret.  

client_id = ""  
client_secret = ""

In [ ]:
# Read the client_secret.json file. If the file is not found or invalid, it will print an error message.
# You can manually input the client ID and client secret if needed.
client_secrets = read_json_file('client_secret.json')

In [ ]:
client_id = client_secrets["web"].get("client_id")
client_secret = client_secrets["web"].get("client_secret")

In [ ]:
os.environ["OAUTH_CLIENT_ID"] = client_id
os.environ["OAUTH_CLIENT_SECRET"] = client_secret
os.environ["OAUTH_TOKEN_URI"] = "https://oauth2.googleapis.com/token"

In [ ]:
%%bash

curl -X POST \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json" \
  -H "X-Goog-User-Project: ${PROJECT_NUMBER}" \
https://discoveryengine.googleapis.com/v1alpha/projects/${PROJECT_NUMBER}/locations/global/authorizations?authorizationId=${AUTH_ID} \
  -d '{
  "name": "projects/${PROJECT_NUMBER}/locations/global/authorizations/${AUTH_ID}",
  "serverSideOauth2": {
      "clientId": "'"${OAUTH_CLIENT_ID}"'",
      "clientSecret": "'"${OAUTH_CLIENT_SECRET}"'",
      "authorizationUri": "'"${OAUTH_AUTH_URI}"'",
      "tokenUri": "'"${OAUTH_TOKEN_URI}"'"
    }
  }'

## 4. Link Agent to Agentspace

In [ ]:
%%bash

curl -X POST \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json" \
  -H "X-Goog-User-Project: ${PROJECT_NUMBER}" \
https://discoveryengine.googleapis.com/v1alpha/projects/${PROJECT_NUMBER}/locations/global/collections/default_collection/engines/${AS_APP}/assistants/default_assistant/agents \
  -d '{
      "displayName": "'"${DISPLAY_NAME}"'",
      "description": "'"${DESCRIPTION}"'",
      "adk_agent_definition": {
        "tool_settings": {
          "tool_description": "'"${TOOL_DESCRIPTION}"'"
        },
        "provisioned_reasoning_engine": {
          "reasoning_engine":
            "'"${REASONING_ENGINE}"'"
        },
        "authorizations": [
          "projects/'"${PROJECT_NUMBER}"'/locations/global/authorizations/'"${AUTH_ID}"'"
        ]
      }
  }'

## 5. Try your agent in Agentspace UI
1. Ensure that `Vertex AI API` and `Discovery Engine API` are enabled in your project and that Discovery Engine service account has `Vertex AI User` permission. If you cannot find it in IAM, check the `Include Google-provided role grants`.
2. Open up the Agentspace app in Google Cloud console
3. Select your Agentspace app
3. Click `Copy URL`
4. Open the URL in a new tab
5. From left menu, select your agent from `Agents` deployed

## 6. List agents (Optional)

In [ ]:
%%bash
# export PROJECT_NUMBER=PROJECT_NUMBER
# export AS_APP=AGENTSAPCE_APP_ID
curl -X GET -H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
-H "X-Goog-User-Project: ${PROJECT_NUMBER}" \
"https://discoveryengine.googleapis.com/v1alpha/projects/${PROJECT_NUMBER}/locations/global/collections/default_collection/engines/${AS_APP}/assistants/default_assistant/agents"

## 7. Delete an agent (when you don't need it anymore )

In [ ]:
os.environ["AGENT_RESOURCE_NAME"] = "your agent resource name printed above"

In [ ]:
%%bash
# export PROJECT_NUMBER=PROJECT_NUMBER
# export AGENT_RESOURCE_NAME=AGENT_RESOURCE_NAME

curl -X DELETE \
  -H "Authorization: Bearer $(gcloud auth print-access-token)" \
  -H "Content-Type: application/json" \
  -H "X-Goog-User-Project: ${PROJECT_NUMBER}" \
https://discoveryengine.googleapis.com/v1alpha/${AGENT_RESOURCE_NAME}